![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F1nVj0%2FbtryniURChE%2FoHO5QsqWiaJNRPjDz3LgzK%2Fimg.png)

---
### 데이터 불러오기
---

In [1]:
from konlpy.tag import Okt
from tqdm import tqdm
import pandas as pd
import numpy as np
from nltk import FreqDist

---
### 수제함수
---

In [2]:
def select_stopword(vocab):
    stop_words = list()
    for word,_ in vocab:
        print(word,': 불용어에 포함시킬 단어면 y 아니면 n 입력, x입력시 취소')
        
        while True:
            input_value = input()
            if input_value in ['y','Y']:
                stop_words.append(word)
                break

            elif input_value in ['n','N']:
                break
            
            elif input_value in ['x','X']:
                return
            else:
                print('잘못된 입력입니다. 다시 입력하시오.')
                continue
    return stop_words

---
### 데이터 확인
---

In [3]:
train_data = pd.read_table('./ratings_train.txt')
test_data = pd.read_table('./ratings_test.txt')

In [4]:
train_data.document = train_data.document.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",regex=True)  #한글을 제외한 단어는 정규식으로 제거

In [5]:
train_data['document'].isna().sum()   # 결측치의 수 확인

5

In [6]:
train_data.dropna(inplace=True)   #결측치 제거

---
### 토큰화
---

[불용어 모음 사이트1](https://www.ranks.nl/stopwords/korean)  
[불용어 모음 사이트2](https://bab2min.tistory.com/544)


In [7]:
stop_word_df1=pd.read_html('https://www.ranks.nl/stopwords/korean')[0]   #페이지의 테이블을 가져온다
stop_word_df2=pd.read_html('https://bab2min.tistory.com/544',encoding='utf-8')[0]

In [8]:
stop_words_list = stop_word_df2.iloc[:,[0,-3]].values.reshape(-1).tolist() # df.values <- 넘파이 배열로 바꿔줌. reshape(-1).1차원 배열로 바꿔줌. tolist() 배열을 리스트로 바꿔줌

In [9]:
for i in range(len(stop_word_df1)):
    words = stop_word_df1.iloc[0,i].split(' ')
    stop_words_list += words
    
stop_words_list += ['너무','다','적','진짜','에서','점','영화','정말','의','가',
                    '이','은','들','는','좀','잘','걍','과','도','를','으로','자',
                    '에','와','한','하다','것','그','안','인','이런','내','못','게',
                    '이다','고','더','때','거','요','본','지','네','입니다','있는',
                    '걸','이렇게','임','라','인데','듯','기','서','난','면','대','라고',
                    '보면','이나','때문','해서','된','엔','두','이고','랑','전','애',
                    '없이','해','속','한다','라는','뿐','되는','니','줄','냐','하게',
                    '하고','합니다','번','하지','보고']  # 수제 모음집

print('불용어 수:',len(stop_words_list))

불용어 수: 463


In [10]:
tokenizer = Okt()   #토큰화에 필요한 Okt 클래스
tokenized = list()  #토큰화된 문장을 담을 리스트 생성
for sentence in tqdm(train_data.document):  # 리뷰 하나씩 꺼내오기. tqdm: iteration 1회에 걸린 시간으로 한 반복문에 걸리는 시간을 추론하여 진행상황을 알려줌
    if not isinstance(sentence,str):          # 이상한 값이 있으면 무시하고 공백 추가
        tokenized.append('')         
    tokenized_sentence = tokenizer.morphs(sentence)                   #문장을 토큰화 리스트 반환
    tokenized_sentence = [word for word in tokenized_sentence if not word in stop_words_list] # 불용어 리스트에 없는 단어만 남기기
    tokenized.append(tokenized_sentence)       #토큰화된 문장을 리스트에 추가

print(len(tokenized))

100%|██████████| 149995/149995 [04:58<00:00, 502.38it/s]

149995


---
### 단어 집합 생성
---

In [11]:
vocab = FreqDist(np.hstack(tokenized))    # 빈도수를 알아서 구해주는 함수와 클래스들
vocab_size = 5000                          # top 5000만 사용하기
vocab = vocab.most_common(vocab_size)     
print(f'단어 집합의 크기 : {len(vocab)}')
print('빈도수 상위 10선:',vocab[:10])

단어 집합의 크기 : 5000
빈도수 상위 10선: [('연기', 6328), ('평점', 6315), ('최고', 6040), ('스토리', 5335), ('드라마', 5063), ('감동', 4881), ('ㅋㅋ', 4307), ('배우', 4292), ('감독', 4118), ('그냥', 4082)]


In [12]:
vocab

[('연기', 6328),
 ('평점', 6315),
 ('최고', 6040),
 ('스토리', 5335),
 ('드라마', 5063),
 ('감동', 4881),
 ('ㅋㅋ', 4307),
 ('배우', 4292),
 ('감독', 4118),
 ('그냥', 4082),
 ('재미', 3921),
 ('내용', 3814),
 ('뭐', 3800),
 ('쓰레기', 3564),
 ('보다', 3552),
 ('없는', 3486),
 ('봤는데', 3033),
 ('작품', 2980),
 ('사랑', 2943),
 ('볼', 2830),
 ('마지막', 2757),
 ('좋은', 2663),
 ('이건', 2605),
 ('같은', 2554),
 ('완전', 2540),
 ('ㅋ', 2538),
 ('ㅠㅠ', 2489),
 ('ㅋㅋㅋ', 2461),
 ('처음', 2453),
 ('장면', 2425),
 ('액션', 2397),
 ('주인공', 2376),
 ('보는', 2294),
 ('최악', 2275),
 ('돈', 2196),
 ('이야기', 2174),
 ('별로', 2132),
 ('봐도', 2123),
 ('느낌', 2102),
 ('참', 2088),
 ('ㅡㅡ', 2082),
 ('연출', 2076),
 ('없고', 2059),
 ('명작', 2041),
 ('끝', 2039),
 ('역시', 1994),
 ('별', 1934),
 ('많이', 1933),
 ('재밌게', 1913),
 ('이해', 1898),
 ('이영화', 1816),
 ('분', 1807),
 ('성', 1734),
 ('아깝다', 1640),
 ('꼭', 1629),
 ('보기', 1613),
 ('짱', 1598),
 ('기억', 1592),
 ('결말', 1580),
 ('ㅎㅎ', 1573),
 ('편', 1570),
 ('같다', 1564),
 ('마음', 1552),
 ('인생', 1542),
 ('영', 1512),
 ('소재', 1508),
 ('넘', 1496)

---
### 인코딩
---

In [13]:
word_to_id = {word[0] : id + 2 for id, word in enumerate(vocab)}
word_to_id['pad'] = 1 
word_to_id['unk'] = 0

encoded = []
for line in tokenized:
    temp = []
    for word in line: 
      try:
        temp.append(word_to_id[word]) 
      except: 
        temp.append(word_to_id['unk'])

    encoded.append(temp)
word_to_id

{'연기': 2,
 '평점': 3,
 '최고': 4,
 '스토리': 5,
 '드라마': 6,
 '감동': 7,
 'ㅋㅋ': 8,
 '배우': 9,
 '감독': 10,
 '그냥': 11,
 '재미': 12,
 '내용': 13,
 '뭐': 14,
 '쓰레기': 15,
 '보다': 16,
 '없는': 17,
 '봤는데': 18,
 '작품': 19,
 '사랑': 20,
 '볼': 21,
 '마지막': 22,
 '좋은': 23,
 '이건': 24,
 '같은': 25,
 '완전': 26,
 'ㅋ': 27,
 'ㅠㅠ': 28,
 'ㅋㅋㅋ': 29,
 '처음': 30,
 '장면': 31,
 '액션': 32,
 '주인공': 33,
 '보는': 34,
 '최악': 35,
 '돈': 36,
 '이야기': 37,
 '별로': 38,
 '봐도': 39,
 '느낌': 40,
 '참': 41,
 'ㅡㅡ': 42,
 '연출': 43,
 '없고': 44,
 '명작': 45,
 '끝': 46,
 '역시': 47,
 '별': 48,
 '많이': 49,
 '재밌게': 50,
 '이해': 51,
 '이영화': 52,
 '분': 53,
 '성': 54,
 '아깝다': 55,
 '꼭': 56,
 '보기': 57,
 '짱': 58,
 '기억': 59,
 '결말': 60,
 'ㅎㅎ': 61,
 '편': 62,
 '같다': 63,
 '마음': 64,
 '인생': 65,
 '영': 66,
 '소재': 67,
 '넘': 68,
 '님': 69,
 '수준': 70,
 '현실': 71,
 '한번': 72,
 '가장': 73,
 '급': 74,
 '매력': 75,
 '반전': 76,
 '전개': 77,
 '함': 78,
 '한국': 79,
 '남자': 80,
 '가슴': 81,
 '아이': 82,
 '음악': 83,
 '원작': 84,
 '만든': 85,
 '화': 86,
 '인간': 87,
 '추천': 88,
 '눈물': 89,
 '않고': 90,
 '인지': 91,
 '모든': 92,
 '봤다': 93,
 '자

In [14]:
id_to_word = {id:word for word,id in word_to_id.items()}
id_to_word

{2: '연기',
 3: '평점',
 4: '최고',
 5: '스토리',
 6: '드라마',
 7: '감동',
 8: 'ㅋㅋ',
 9: '배우',
 10: '감독',
 11: '그냥',
 12: '재미',
 13: '내용',
 14: '뭐',
 15: '쓰레기',
 16: '보다',
 17: '없는',
 18: '봤는데',
 19: '작품',
 20: '사랑',
 21: '볼',
 22: '마지막',
 23: '좋은',
 24: '이건',
 25: '같은',
 26: '완전',
 27: 'ㅋ',
 28: 'ㅠㅠ',
 29: 'ㅋㅋㅋ',
 30: '처음',
 31: '장면',
 32: '액션',
 33: '주인공',
 34: '보는',
 35: '최악',
 36: '돈',
 37: '이야기',
 38: '별로',
 39: '봐도',
 40: '느낌',
 41: '참',
 42: 'ㅡㅡ',
 43: '연출',
 44: '없고',
 45: '명작',
 46: '끝',
 47: '역시',
 48: '별',
 49: '많이',
 50: '재밌게',
 51: '이해',
 52: '이영화',
 53: '분',
 54: '성',
 55: '아깝다',
 56: '꼭',
 57: '보기',
 58: '짱',
 59: '기억',
 60: '결말',
 61: 'ㅎㅎ',
 62: '편',
 63: '같다',
 64: '마음',
 65: '인생',
 66: '영',
 67: '소재',
 68: '넘',
 69: '님',
 70: '수준',
 71: '현실',
 72: '한번',
 73: '가장',
 74: '급',
 75: '매력',
 76: '반전',
 77: '전개',
 78: '함',
 79: '한국',
 80: '남자',
 81: '가슴',
 82: '아이',
 83: '음악',
 84: '원작',
 85: '만든',
 86: '화',
 87: '인간',
 88: '추천',
 89: '눈물',
 90: '않고',
 91: '인지',
 92: '모든',
 93: '봤다',
 94

In [15]:
encoded

[[294, 0, 477],
 [735, 295, 422, 1353, 2, 781, 0, 0],
 [241, 2625, 0, 2455, 0, 0, 88],
 [0, 37, 0, 101, 12, 3, 4144],
 [844, 0, 0, 2, 0, 2784, 0, 0, 314, 0, 0, 910, 3416, 4326],
 [542, 0, 798, 0, 276, 1659, 1576, 0, 0, 29, 48, 911, 410, 103],
 [84, 175, 186, 0, 110],
 [48,
  911,
  55,
  194,
  567,
  0,
  0,
  157,
  1520,
  91,
  621,
  231,
  341,
  0,
  3479,
  0,
  1285,
  1285,
  6,
  138,
  2,
  518,
  904],
 [32, 0, 12, 255],
 [1455, 3, 0, 360, 21, 1342, 218, 0, 0, 923],
 [0, 58, 58],
 [568, 240, 0, 0, 229, 2717, 203, 0, 426, 2676, 627, 0],
 [2372, 671, 0, 0, 4145, 0, 578, 597, 4233, 2, 1277],
 [0, 0, 104, 309, 1051, 547, 1380, 16, 712, 0, 0],
 [685, 4529, 0, 1785, 189, 73, 191, 858, 7, 5, 2423, 7, 2423],
 [1670, 1074, 2755, 1267, 511, 2070, 28],
 [41,
  0,
  0,
  1140,
  0,
  447,
  0,
  0,
  1140,
  1295,
  230,
  0,
  2455,
  1074,
  0,
  0,
  100,
  1226],
 [0, 0, 1034, 51, 367, 320, 338, 12, 0],
 [24, 2650, 296, 0, 163, 0, 0, 0, 2650, 4428],
 [0, 165, 0, 131, 208, 1644, 20

In [18]:
maximum_length = max(list(map(len,encoded)))

In [20]:
len(encoded)

149995

In [19]:
maximum_length

67